In [ ]:

import pickle
import glob
import os
from tqdm import tqdm
import numpy as np
import copy
current_directory = os.getcwd()

if 'experiments' in current_directory:
    parent_directory = os.path.dirname(current_directory)
    os.chdir(parent_directory)
num_classes = 19
exp = 'table'

def read_exp_dir(ds, exp='table'):
    exp_dir = f'log/{ds}/{exp}'
    overall_dict = {}
    for file in tqdm(glob.glob(os.path.join(exp_dir, '*.pkl'))[:100], desc=f'reading from {exp_dir}'):
        d = pickle.load(open(file, 'rb'))
        filename = os.path.basename(file).replace('.pkl', '')
        new_d = {}
        new_d[filename] = d[filename]
        d = new_d
        for image_name, image_d in d.items():
            for model_type, model_d in image_d.items():
                if model_type not in overall_dict:
                    overall_dict[model_type] = {}
                for metric, value in model_d.items():
                    if isinstance(value, dict):
                        for k, v in value.items():
                            if isinstance(v, dict):
                                if metric not in overall_dict[model_type]:
                                    overall_dict[model_type][metric] = {}
                                if k not in overall_dict[model_type][metric]:
                                    overall_dict[model_type][metric][k] = {}
                                for k_, v_ in v.items():
                                    if k_ not in overall_dict[model_type][metric][k]:
                                        overall_dict[model_type][metric][k][k_] = v_
                                    else:
                                        overall_dict[model_type][metric][k][k_] += v_
                            else:
                                if k not in overall_dict[model_type]:
                                    overall_dict[model_type][k] = v
                                else:
                                    overall_dict[model_type][k] += v
                                
                        continue
                    if metric not in overall_dict[model_type]:
                        overall_dict[model_type][metric] = value
                    else:
                        overall_dict[model_type][metric] += value
    return overall_dict

dir_dict = {}
for ds in ['cityscapes', 'acdc', 'cocostuff', 'pascal_ctx']:
    dir_dict[ds] = read_exp_dir(ds)

In [ ]:
col_d = {}

dir_dict_ = copy.deepcopy(dir_dict)
num_classes = { 'cocostuff': 171, 'pascal_ctx': 59, 'cityscapes': 19, 'acdc': 19}

for ds, ds_d in dir_dict_.items():
    col_d[ds] = {}
    for model_type, model_d in ds_d.items():
        confusion_matrix = model_d['confusion_matrix']
        pos = confusion_matrix.sum(1)
        res = confusion_matrix.sum(0)
        tp = np.diag(confusion_matrix)
        pixel_acc = tp.sum()/pos.sum()
        mean_acc = (tp/np.maximum(1.0, pos)).mean()
        IoU_array = (tp / np.maximum(1.0, pos + res - tp))
        mean_IoU = IoU_array.mean()
        if isinstance(model_type, tuple):
            n, n0, f, h, sigma, tau = model_type
            if isinstance(f, str):
                col_d[ds] = f
                mean_IoU = '-'
            assert sum(model_d['num_pixels_per_cls']) == model_d['num_pixels']
            dir_dict[ds][model_type]['CIG'] = sum(model_d['cig_per_cls'])/sum(model_d['num_pixels_per_cls'])/np.log(num_classes[ds])
            dir_dict[ds][model_type]['cCIG'] = np.mean([c/n/np.log(num_classes[ds]) for c, n in zip(model_d['cig_per_cls'], model_d['num_pixels_per_cls']) if n > 0])
            dir_dict[ds][model_type]['PC'] = model_d['certified_count']/model_d['num_pixels']
            dir_dict[ds][model_type]['cPC'] = np.mean([c/n for c, n in zip(model_d['certified_per_cls'], model_d['num_pixels_per_cls']) if n > 0])
            dir_dict[ds][model_type]['abstain'] = model_d['abstain_count']/model_d['num_pixels']
            dir_dict[ds][model_type]['cabstain'] = np.mean([c/n for c, n in zip(model_d['abstain_per_cls'], model_d['num_pixels_per_cls']) if n > 0])
            dir_dict[ds][model_type]['mIoU'] = mean_IoU   
        else:
            model_type = (None, None, None, 0, None, None) 
            dir_dict[ds][model_type] = {'CIG' :pixel_acc,
                                        'cCIG':mean_acc,
                                        'mIoU':mean_IoU,          
                                        'cPC':'-',
                                        'PC':'-',
                                        'cabstain':'-',
                                        'abstain':'-'}


In [ ]:
    
col = [
    (None, None, None, 0, None, None),
    (100, 10, None, 0, 0.25, 0.75),
    (100, 10, None, 0, 0.33, 0.75),
    (100, 10, None, 0, 0.5, 0.75),

    (100, 10, '', 4, 0.25, 0.75),
    (100, 10, '', 4, 0.33, 0.75),
    (100, 10, '', 4, 0.5, 0.75),
    
    (500, 10, None, 0, 0.25, 0.95),
    (500, 10, None, 0, 0.33, 0.95),
    (500, 10, None, 0, 0.5, 0.95),

    (500, 10, '', 4, 0.25, 0.95),
    (500, 10, '', 4, 0.33, 0.95),
    (500, 10, '', 4, 0.5, 0.95),
]

def edit_table(model_k, sub_k, graph_dicts, str_to_replace, table):
    
    val = graph_dicts[ds][model_k][sub_k]
    n, n0, th_func, i, sigma, tau = model_k
    val_seg = graph_dicts[ds][(n, n0, None, 0, sigma, tau)][sub_k]

    if val != '-':
        if sub_k == 'CIG' or sub_k == 'cCIG': 
            val = round(val, 2)
            val_seg = round(val_seg, 2)
        elif sub_k == 'abstain' or sub_k == 'cabstain': 
            val = int(round(val*100))
            val_seg = int(round(val_seg*100))

    ext = ''
    if th_func != None:
        if  sub_k == 'cCIG' or sub_k == 'CIG':
            cig_adapt = val
            cig_seg = val_seg
            if cig_adapt > cig_seg:
                diff = cig_adapt - cig_seg if cig_seg == 0 else (cig_adapt-cig_seg)/cig_seg
                ext  = r'{\fontsize{2}{0}\selectfont\textcolor{ForestGreen}{$'+'{0:.1f}'.format(diff*100)+r'\%$}}'
            elif cig_adapt < cig_seg:
                diff = cig_adapt - cig_seg if cig_seg == 0 else (cig_adapt-cig_seg)/cig_seg
                ext  = r'{\fontsize{2}{0}\selectfont\textcolor{red}{$'+'{0:.1f}'.format(diff*100)+r'\%$}}'
        if sub_k =='cabstain' or sub_k == 'abstain':
            abstain_adapt = val
            abstain_seg = val_seg
            if abstain_adapt > abstain_seg:
                ext  = r'{\fontsize{2}{0}\selectfont\textcolor{red}{$'+'{0:.1f}'.format((abstain_adapt-abstain_seg)/abstain_seg*100)+r'\%$}}'
            elif abstain_adapt < abstain_seg:
                ext  = r'{\fontsize{2}{0}\selectfont\textcolor{ForestGreen}{$'+'{0:.1f}'.format((abstain_seg-abstain_adapt)/abstain_seg*100)+r'\%$}}'
    if (sub_k == 'abstain'  or sub_k == 'cabstain') and val != '-':
        val = '{0:.0f}'.format(val)
    elif val != '-':
        val = '{0:.2f}'.format(val)
    if sub_k == 'mIoU':
        table = table.replace(f'{str_to_replace}', f'${val}$')
    if th_func != None:
        table = table.replace(f'{str_to_replace}',r'\textbf{'+ f'{val} {ext}'+'}')
    else:
        table = table.replace(f'{str_to_replace}', f'${val}$')
    return table

def edit_col(st, ds, key, graph_dicts, table):
    row_len = 5-1
    vs = ['$v{:02d}$'.format(i+st+i*row_len) if i+st+i*row_len < 100 else '$vh{:02d}$'.format(i+st+i*row_len) for i in range(13)]
    for model_k, v in zip(col, vs):
        n, n0, th, hi, sigma, tau = model_k
        if th != None:
            th = col_d[ds]
        model_k = (n, n0, th, hi, sigma, tau)
        table = edit_table(model_k, key, graph_dicts, v, table)
    return table


ds_formal = {'pascal_ctx': 'PASCAL-Context', 'cocostuff': 'COCO-Stuff-10k', 'cityscapes':'Cityscapes', 'acdc':'ACDC'}
for ds in ['acdc', 'cityscapes', 'pascal_ctx', 'cocostuff']:
    table = r"""\begin{table*}[!ht]
    \caption{Certified segmentation results on the first 100 images in  DATASET.}
    \label{tab:dataset-table}
    %\setlength\tabcolsep{3.6pt}
    \begin{tabularx}{\textwidth}{lXrrllllll}
    \toprule
    \multicolumn{5}{l}{} & \multicolumn{5}{c}{DATASET} \\
    \cmidrule{6-10}
    &  & \multicolumn{1}{l}{$\sigma$} & \multicolumn{1}{l}{$R$} &  & $\cig\uparrow$ & $c\cig\uparrow$  & $\%\oslash\downarrow$ &  $c\%\oslash\downarrow$ & mIoU $\uparrow$ \\
    \midrule
    & Uncertified HrNet & \multicolumn{1}{l}{-} & \multicolumn{1}{l}{-} &  & $v01$ & $v02$ & $v03$ & $v04$ & $v05$ \\
    \midrule
    \multirow{7}{*}{\begin{tabular}[c]{@{}l@{}}$n = 100,$\\ $\tau=0.75$\end{tabular}} & \multirow{3}{*}{\textsc{SegCertify}} & $0.25$ & $0.17$ &  & $v06$ & $v07$ & $v08$ & $v09$ & $v10$ \\
    &  & $0.33$ & $0.22$ &  & $v11$ & $v12$ & $v13$ & $v14$ & $v15$ \\
    &  & 0.50 & 0.34 &  & $v16$ & $v17$ & $v18$ & $v19$ & $v20$ \\
    & \multicolumn{8}{l}{} \\
    & \multirow{3}{*}{\textsc{AdaptiveCertify}} & 0.25 & 0.17 &  & $v21$ & $v22$ & $v23$ & $v24$ & $v25$ \\
    &  & 0.33 & 0.22 &  & $v26$ & $v27$ & $v28$ & $v29$ & $v30$ \\
    &  & 0.50 & 0.34 &  & $v31$ & $v32$ & $v33$ & $v34$ & $v35$ \\
    \midrule
    \multirow{7}{*}{\begin{tabular}[c]{@{}l@{}}$n=500,$\\ $\tau=0.95$\end{tabular}} & \multirow{3}{*}{\textsc{SegCertify}} & 0.25 & 0.41 &  & $v36$ & $v37$ & $v38$ & $v39$ & $v40$ \\
    &  & 0.33 & 0.52 &  & $v41$ & $v42$ & $v43$ & $v44$ & $v45$ \\
    &  & 0.50 & 0.82 &  & $v46$ & $v47$ & $v48$ & $v49$ & $v50$ \\
    & \multicolumn{8}{l}{} \\
    & \multirow{3}{*}{\textsc{AdaptiveCertify}} & 0.25 & 0.41 &  & $v51$ & $v52$ & $v53$ & $v54$ & $v55$ \\
    &  & 0.33 & 0.52 &  & $v56$ & $v57$ & $v58$ & $v59$ & $v60$ \\
    &  & 0.50 & 0.82 &  & $v61$ & $v62$ & $v63$ & $v64$ & $v65$ \\
    \bottomrule
    \end{tabularx}
    \end{table*}
    """
    table = table.replace('DATASET', ds_formal[ds])
    table = table.replace('dataset', ds)
    table = edit_col(1, ds, 'CIG', dir_dict, table)
    table = edit_col(2, ds, 'cCIG', dir_dict, table)
    table = edit_col(3, ds, 'abstain', dir_dict, table)
    table = edit_col(4, ds, 'cabstain', dir_dict, table)
    table = edit_col(5, ds, 'mIoU', dir_dict, table)
    text_file = open(f"graph_images/{ds}.tex", "w")
    text_file.write(table)
    text_file.close()


In [ ]:
table_paper = r"""\begin{table*}[bp]
\caption{Certified segmentation results for 100 images from each dataset. $\cig$ and $c\cig$ stand for per-pixel certified info. gain and class-average certified info. gain, respectively. $\%\oslash$ is the abstain rate, and $c\%\oslash$ is the class-average abstain rate.  }
\label{tab:table}
\setlength\tabcolsep{2pt}
\begin{tabularx}{\textwidth}{lXrrlllllllll}
\toprule
\multicolumn{5}{l}{} & \multicolumn{2}{c}{DATASET1} & \multicolumn{2}{c}{DATASET2} & \multicolumn{2}{c}{DATASET3} & \multicolumn{2}{c}{DATASET4} \\
\cmidrule{6-13}
&  & \multicolumn{1}{l}{$\sigma$} & \multicolumn{1}{l}{$R$} &  & $\cig\uparrow$ & $\%\oslash\downarrow$  & $\cig\uparrow$ & $\%\oslash\downarrow$ & $\cig\uparrow$ & $\%\oslash\downarrow$ & $\cig\uparrow$ & $\%\oslash\downarrow$\\
\midrule
& Uncert. HrNet & \multicolumn{1}{l}{-} & \multicolumn{1}{l}{-} &  & $v01$ & $v02$ & $v03$ & $v04$ & $v05$ & $v06$ &  $v07$ & $v08$ \\
\midrule
\multirow{7}{*}{\begin{tabular}[c]{@{}l@{}}$n = 100,$\\ $\tau=0.75$\end{tabular}} & \multirow{3}{*}{\textsc{SegCertify}} & $0.25$ & $0.17$ &  & $v09$ & $v10$ & $v11$ & $v12$ & $v13$ & $v14$ & $v15$ & $v16$ \\
&  & $0.33$ & $0.22$ &  & $v17$ & $v18$ & $v19$ & $v20$ & $v21$ & $v22$ & $v23$ & $v24$ \\
&  & 0.50 & 0.34 &   & $v25$ & $v26$ & $v27$ & $v28$ & $v29$ & $v30$ & $v31$ & $v32$ \\
& \multicolumn{8}{l}{} \\
& \multirow{3}{*}{\textsc{AdaptiveCertify}} & 0.25 & 0.17 &  & $v33$ & $v34$ & $v35$ & $v36$ & $v37$ & $v38$ & $v39$ & $v40$ \\
&  & 0.33 & 0.22 &  & $v41$ & $v42$ & $v43$ & $v44$ & $v45$ & $v46$ & $v47$ & $v48$ \\
&  & 0.50 & 0.34 &  & $v49$ & $v50$ & $v51$ & $v52$ & $v53$ & $v54$ & $v55$ & $v56$ \\
\midrule
\multirow{7}{*}{\begin{tabular}[c]{@{}l@{}}$n=500,$\\ $\tau=0.95$\end{tabular}} & \multirow{3}{*}{\textsc{SegCertify}} & 0.25 & 0.41 &  & $v57$ & $v58$ & $v59$ & $v60$ & $v61$ & $v62$ & $v63$ & $v64$\\
&  & 0.33 & 0.52 &  & $v65$ & $v66$ & $v67$ & $v68$ & $v69$ & $v70$ & $v71$ & $v72$\\
&  & 0.50 & 0.82 &  & $v73$ & $v74$ & $v75$ & $v76$ & $v77$ & $v78$ & $v79$ & $v80$ \\
& \multicolumn{8}{l}{} \\
& \multirow{3}{*}{\textsc{AdaptiveCertify}} & 0.25 & 0.41 &  & $v81$ & $v82$ & $v83$ & $v84$ & $v85$ & $v86$ & $v87$ & $v88$\\
&  & 0.33 & 0.52 &  & $v89$ & $v90$ & $v91$ & $v92$ & $v93$ & $v94$ & $v95$ & $v96$\\
&  & 0.50 & 0.82 &  & $v97$ & $v98$ & $v99$ & $v100$ & $v101$ & $v102$ & $v103$ & $v104$\\
\bottomrule
\end{tabularx}
\end{table*}
"""

table = table_paper
def edit_col(col_idx, ds, key, graph_dicts, table):
    row_len = 7
    vs = ['$v{:02d}$'.format(i+1+col_idx+ i*row_len) for i in range(13)]
    print(vs)
    for model_k, v in zip(col, vs):
        n, n0, th, hi, sigma, tau = model_k
        if th != None:
            th = col_d[ds]
        model_k = (n, n0, th, hi, sigma, tau)
        table = edit_table(model_k, key, graph_dicts, v, table)
    return table

ds = 'acdc'
ds_formal = {'pascal_ctx': 'PASCAL-Context', 'cocostuff': 'COCO-Stuff-10K', 'cityscapes':'Cityscapes', 'acdc':'ACDC'}
for i, ds in enumerate(['cityscapes', 'acdc', 'pascal_ctx', 'cocostuff']):
    table = table.replace(f'DATASET{i+1}', ds_formal[ds])
    table = edit_col(i*2, ds, 'CIG', dir_dict, table)
    table = edit_col(i*2+1, ds, 'abstain', dir_dict, table)

text_file = open(f"graph_images/table_paper.tex", "w")
text_file.write(table)
text_file.close()
